In [1]:
import import_ipynb
import LexicalAnalysis as la
from sys import stdout, modules

importing Jupyter notebook from LexicalAnalysis.ipynb


In [2]:
class Stack:
    '''Data structure for a Stack (LIFO)'''

    def __init__(self, another=None):
        self.list = []
        if another:
            self.list = another.list

    def push(self, val):
        self.list.append(val)

    def pop(self):
        return self.list.pop() if self.list else None

    def peek(self):
        return self.list[-1] if self.list else None

    def isEmpty(self):
        return False if self.list else True

    def size(self):
        return len(self.list)
    
    def reverse(self):
        return self.list.reverse()
    
    def display(self):
        #print(self.list)
        return self.list
    
    def displayElements(self):
        print(len(self.list))
        #prints list elements on each line rather than whole list at once
        for element in range(0,len(self.list)):
            print(self.list[element])
            print(self.list[element].token)
            print(self.list[element].left)
            print(self.list[element].right)
            #print(self.list[element].type)
            #print(self.list[element].value)
            

In [16]:
class AST:
    def __init__(self, token, left, right):
        self.value = token.value
        self.left = left
        self.right = right
    
    def preOrder(self):
        # Value, Left, Right
        result = []
        #result.append("(")
        result.append(self.value)
        if self.left:
            result.append("(")
            result += self.left.preOrder()
            if not self.right:
                result.append(")")
                
        if self.right:
            if self.left:
                result.append(",")
            else: 
                result.append("(")
            result += self.right.preOrder()
            result.append(")")
        #result.append(")")
        return result

    def inOrder(self):
        # Left, Value, Right
        result = []
        if self.left:
            result += self.left.inOrder()
        result.append(self.value)
        if self.right:
            result += self.right.inOrder()
        return result

    def postOrder(self):
        # Left, Right, Value
        result = []
        if self.left:
            result += self.left.postOrder()
        if self.right:
            result += self.right.postOrder()
        result.append(self.value)
        return result

    def _format(self, _padding=''):
        #Formats the AST to a string

        line = '('+str(self.value)+')\n'

        if self.left:
            line += _padding
            line += ' \u251C'

            # push
            _padding += ' |'

            line += self.left._format(_padding)

            # pop
            _padding = _padding[:-2]

        if self.right:
            line += _padding
            line += ' \u2514'
            
            # push
            _padding += '  '

            line += self.right._format(_padding)

            # pop
            _padding = _padding[:-2]
        return line 

    def format(self):
        return self._format()

    def display(self):
        #Pretty-prints the AST to the terminal
        #print(self.right)
        #print(self.value)
        #print(self.left)
        stdout.write(self.format())

    def printPreOrder(self):
        stdout.write(' '.join(map(str, self.preOrder()))+'\n')

    def printInOrder(self):
        stdout.write(' '.join(map(str, self.inOrder()))+'\n')

    def printPostOrder(self):
        stdout.write(' '.join(map(str, self.postOrder()))+'\n')
        
    
        
    def printString(self):
        A = (''.join(map(str, self.preOrder())))
        return A


In [17]:
def addNodeToAST(stack, operatorToken, isFunction=False):
    rightChild = stack.pop()
    #print(dealingWithFunc)
    if isFunction == False:
        leftChild = stack.pop()
    else:
        leftChild = None

    stack.push(AST(operatorToken, leftChild, rightChild))
    #print(leftChild, rightChild)
    

In [18]:
def Parse(inp, tokens):
    #outQueue = []
    outStack = Stack()
    opStack = Stack()
    result = ""
    #global dealingWithFunc
    #global isBracket
    
    ASSOCIATIVITY = {
        '^' : "right", # raise
        '*' : "left", # multiply
        '/' : "left", # divide
        '+' : "left", # add
        '-' : "left"  # subtract
    }

    PRECEDENCE = {
        '^' : 4, # raise
        '*' : 3, # multiply
        '/' : 3, # divide
        '+' : 2, # add
        '-' : 2  # subtract
    }

    SYMPY_EQUIV = {
        '^' : "Pow", # raise
        '*' : "Mul", # multiply
        '/' : "Div", # divide
        '+' : "Add", # add
        '-' : "Sub"  # subtract
    }
    
    #tokens = la.Tokenise(inp)
    
    print(len(tokens), "tokens.")
    #print(opStack.size())
    
    for t in tokens:
        if(t.type == "Operator"):
            t.precedence = PRECEDENCE[t.value]
            t.associativity = ASSOCIATIVITY[t.value]
            # 
            t.value = SYMPY_EQUIV[t.value]
    
    for t in tokens:
        if(t.type == "Literal" or t.type == "Variable"):
            #outQueue.append(t)
            outStack.push(AST(t, None, None))
        elif(t.type == "Function"):
            opStack.push(t)
            #addNodeToAST(outStack, opStack.pop())
        elif(t.type == "Operator"):

            #if(opStack.isEmpty() == False):
                #print(opStack.peek().type)
                #print(t.associativity)
                #print(t.precedence())
                #print(opStack.peek().precedence())
                #while(opStack.peek().type == "Operator") and ((t.associativity() == "left" and t.precedence() <= opStack.peek().precedence()) or (t.associativity() == "right" and t.precedence() < opStack.peek().precedence()):
            while(opStack.peek() and opStack.peek().type == "Operator") and ((t.associativity == "left") and (t.precedence <= opStack.peek().precedence) or (t.associativity == "right") and (t.precedence < opStack.peek().precedence)):
                    #outQueue.append(opStack.pop()
                addNodeToAST(outStack, opStack.pop())
            opStack.push(t)
        elif(t.type == "Left Parenthesis"):
            opStack.push(t)
        elif(t.type == "Right Parenthesis"):
            #if(opStack.isEmpty() == False):#
            #dealingWithFunc = True
            while(opStack.peek() and opStack.peek().type != "Left Parenthesis"):
                    #outQueue.append(opStack.pop())
                ###
                #isBracket = True
                #opStack.pop()
                #opStack.pop()
                ###
                addNodeToAST(outStack, opStack.pop())
            opStack.pop()
                
            if((opStack.isEmpty() == False) and (opStack.peek().type == "Function")):
                    #outQueue.append(opStack.pop())
                addNodeToAST(outStack, opStack.pop(), True)
                    
            #dealingWithFunc = False
    #if(opStack.size()>1):
    #    opStack.reverse()
    #    outQueue = outQueue + opStack.display()
    #elif(opStack.size()>0):
     #   outQueue.append(opStack.peek())

    #for i in outQueue:
    #    result = result + (i.value) + " "
        
        
    #print(result)
    
    while(opStack.peek()):
        addNodeToAST(outStack, opStack.pop())
        
    #print(outStack.displayElements())
    return outStack.pop()
         

In [19]:
def hasVars(tokens):
    for t in tokens:
        if(t.type == "Variable"):
            return True
            break

In [20]:
#Parse("3 + 4 * 2 / ( 1 - 5 ) ^ 2 ^ ")
#t = Parse("sin(2x)")
#t=Parse("9+3+-7")
#t= Parse("-4+6")
#t= Parse("-cos(1)")
#t= Parse("-8*-99")
#t = Parse("-sin(-cos(tan(1)))")

#t = Parse("-sin(-cos(9))")
#t = Parse("1+sin(99)")
#t = Parse("--cos(56)")
#t = Parse("sqrt(-5sin(2.6))*cos(2)")

#t = Parse("-4.5+6sin(-8.79)")
#t = Parse("2*(9+5)")

#t = Parse("cos(10)+sin(99)")
#t = Parse("sin(99)+1")
#t = Parse("sin(20)/cos(45)")
#t = Parse("sin(30)*sin(30)")

#t = Parse("-3x+(4-x)")
#t = Parse("3x/5")
#t = Parse("(4x)/(9y)")
#t = Parse("x^2+4x")
#t = Parse("(sqrt(4x))^2")

#t = Parse("8(9+10)")
#t = Parse("5ahb")

#t = Parse("-cos(8)")

#t = Parse("1+7*9")
#t = Parse("-9 + 3 * 6")
#t = Parse("-sin(9)")
#t = Parse("-3sqrt(sqrt(sin(25)/cos(20)*tan(sin(10/2))))")
#t = Parse("3x^2+6x+4")
#t.printPreOrder()
#t.printInOrder()
#t.printPostOrder()  
#t.display()
#t.printString()
 
#tokens = la.Tokenise("5+4")
#a = Parse(expression, tokens)
#a.printString()


#expression = "5+4"
#tokens = la.Tokenise(expression)
#a = Parse(expression, tokens)

#expression = a.printString()
#print(expression)

    
#xpression = a.printString()
#rint(expression)

#c.Calc("5+4")

Literal
5
Operator
+
Literal
4
3 tokens.
Add(5,4)
